In [ ]:
# Tạ Cao Sơn - B22DCVT445
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Download NLTK data
nltk.download('stopwords', quiet=True)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Tạ Cao Sơn - B22DCVT445

# Load dataset 1: combined_dataset.csv
df1 = pd.read_csv('combined_dataset.csv')
df1.columns = ['label', 'text']
print(f"Dataset 1 shape: {df1.shape}")
print(f"Dataset 1 head:\n{df1.head()}\n")

# Load dataset 2: spam_ham_dataset.csv
df2 = pd.read_csv('spam_ham_dataset.csv')
df2 = df2[['label', 'text']]
print(f"Dataset 2 shape: {df2.shape}")
print(f"Dataset 2 head:\n{df2.head()}\n")

# Load dataset 3: spam.csv
df3 = pd.read_csv('spam.csv', encoding='latin-1')
df3 = df3.iloc[:, :2]  # Only take first 2 columns
df3.columns = ['label', 'text']
print(f"Dataset 3 shape: {df3.shape}")
print(f"Dataset 3 head:\n{df3.head()}\n")

# Merge all datasets
df = pd.concat([df1, df2, df3], ignore_index=True)
print(f"\nMerged dataset shape: {df.shape}")
print(f"Label distribution:\n{df['label'].value_counts()}")

In [ ]:

# Check for missing values
print("Missing values:")
print(df.isnull().sum())

# Remove duplicates
df = df.drop_duplicates()
print(f"\nAfter removing duplicates: {df.shape}")

# Visualize label distribution
plt.figure(figsize=(8, 5))
df['label'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.title('Distribution of Spam vs Ham')
plt.xlabel('Label')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

# Text length distribution
df['text_length'] = df['text'].apply(len)
print(f"\nText length statistics:")
print(df.groupby('label')['text_length'].describe())

In [ ]:
# Tạ Cao Sơn - B22DCVT445
def preprocess_text(text):
    """
    Tiền xử lý văn bản:
    - Chuyển về chữ thường
    - Loại bỏ số
    - Loại bỏ dấu câu
    - Loại bỏ stopwords
    - Stemming
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    # Remove stopwords and apply stemming
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    
    return ' '.join(words)

# Apply preprocessing
print("Preprocessing text data...")
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Show examples
print("\nOriginal vs Cleaned examples:")
for i in range(3):
    print(f"\nOriginal: {df['text'].iloc[i][:100]}...")
    print(f"Cleaned: {df['cleaned_text'].iloc[i][:100]}...")

In [ ]:
# Tạ Cao Sơn - B22DCVT445

# Convert labels to binary (spam=1, ham=0)
df['label_binary'] = df['label'].map({'spam': 1, 'ham': 0})

# Split features and labels
X = df['cleaned_text'].values
y = df['label_binary'].values

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")
print(f"Training label distribution: {np.bincount(y_train)}")
print(f"Testing label distribution: {np.bincount(y_test)}")

In [ ]:
# Tạ Cao Sơn - B22DCVT445

# Tokenization parameters
MAX_WORDS = 10000
MAX_LEN = 200
EMBEDDING_DIM = 128

# Initialize tokenizer
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

print(f"Vocabulary size: {len(tokenizer.word_index)}")
print(f"Training data shape: {X_train_pad.shape}")
print(f"Testing data shape: {X_test_pad.shape}")
print(f"\nExample sequence: {X_train_pad[0][:20]}")

In [ ]:
def build_dense_model():
    """
    Mô hình Dense với 7 layers:
    1. Embedding layer
    2. GlobalMaxPooling1D
    3-7. Dense layers với Dropout
    """
    model = Sequential([
        # Layer 1: Embedding
        Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
        
        # Layer 2: Global Max Pooling
        GlobalMaxPooling1D(),
        
        # Layer 3: Dense layer
        Dense(256, activation='relu'),
        Dropout(0.5),
        
        # Layer 4: Dense layer
        Dense(128, activation='relu'),
        Dropout(0.4),
        
        # Layer 5: Dense layer
        Dense(64, activation='relu'),
        Dropout(0.3),
        
        # Layer 6: Dense layer
        Dense(32, activation='relu'),
        Dropout(0.2),
        
        # Layer 7: Output layer
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Build Dense model
dense_model = build_dense_model()
dense_model.summary()

In [ ]:
# Tạ Cao Sơn - B22DCVT445

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

# Train Dense model
print("Training Dense model...")
history_dense = dense_model.fit(
    X_train_pad, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

In [ ]:
# Tạ Cao Sơn - B22DCVT445

# Evaluate Dense model
y_pred_dense = (dense_model.predict(X_test_pad) > 0.5).astype(int)

print("="*50)
print("DENSE MODEL EVALUATION")
print("="*50)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred_dense):.4f}")
print(f"\nClassification Report:\n{classification_report(y_test, y_pred_dense, target_names=['Ham', 'Spam'])}")

# Confusion Matrix
cm_dense = confusion_matrix(y_test, y_pred_dense)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_dense, annot=True, fmt='d', cmap='Blues', xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title('Confusion Matrix - Dense Model')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(history_dense.history['accuracy'], label='Train Accuracy')
axes[0].plot(history_dense.history['val_accuracy'], label='Val Accuracy')
axes[0].set_title('Dense Model - Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(history_dense.history['loss'], label='Train Loss')
axes[1].plot(history_dense.history['val_loss'], label='Val Loss')
axes[1].set_title('Dense Model - Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def build_lstm_model():
    """
    Mô hình LSTM với 7 layers:
    1. Embedding layer
    2. Bidirectional LSTM
    3. LSTM layer
    4-6. Dense layers với Dropout
    7. Output layer
    """
    model = Sequential([
        # Layer 1: Embedding
        Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_LEN),
        
        # Layer 2: Bidirectional LSTM
        Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        
        # Layer 3: LSTM layer
        LSTM(64, dropout=0.2, recurrent_dropout=0.2),
        
        # Layer 4: Dense layer
        Dense(128, activation='relu'),
        Dropout(0.5),
        
        # Layer 5: Dense layer
        Dense(64, activation='relu'),
        Dropout(0.4),
        
        # Layer 6: Dense layer
        Dense(32, activation='relu'),
        Dropout(0.3),
        
        # Layer 7: Output layer
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Build LSTM model
lstm_model = build_lstm_model()
lstm_model.summary()

In [ ]:
# Train LSTM model
print("Training LSTM model...")
history_lstm = lstm_model.fit(
    X_train_pad, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

In [ ]:
# Evaluate LSTM model
y_pred_lstm = (lstm_model.predict(X_test_pad) > 0.5).astype(int)

print("="*50)
print("LSTM MODEL EVALUATION")
print("="*50)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred_lstm):.4f}")
print(f"\nClassification Report:\n{classification_report(y_test, y_pred_lstm, target_names=['Ham', 'Spam'])}")

# Confusion Matrix
cm_lstm = confusion_matrix(y_test, y_pred_lstm)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_lstm, annot=True, fmt='d', cmap='Greens', xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title('Confusion Matrix - LSTM Model')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(history_lstm.history['accuracy'], label='Train Accuracy')
axes[0].plot(history_lstm.history['val_accuracy'], label='Val Accuracy')
axes[0].set_title('LSTM Model - Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(history_lstm.history['loss'], label='Train Loss')
axes[1].plot(history_lstm.history['val_loss'], label='Val Loss')
axes[1].set_title('LSTM Model - Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Compare model performance
comparison_data = {
    'Model': ['Dense', 'LSTM'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_dense),
        accuracy_score(y_test, y_pred_lstm)
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("="*50)
print("MODEL COMPARISON")
print("="*50)
print(comparison_df.to_string(index=False))

# Bar chart comparison
plt.figure(figsize=(10, 6))
plt.bar(comparison_df['Model'], comparison_df['Accuracy'], color=['blue', 'green'], alpha=0.7)
plt.ylim(0.9, 1.0)
plt.ylabel('Accuracy')
plt.title('Model Comparison: Dense vs LSTM')
plt.grid(axis='y', alpha=0.3)

for i, v in enumerate(comparison_df['Accuracy']):
    plt.text(i, v + 0.002, f'{v:.4f}', ha='center', fontweight='bold')

plt.show()

# Training history comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(history_dense.history['accuracy'], label='Dense Train', linestyle='-', color='blue')
axes[0].plot(history_dense.history['val_accuracy'], label='Dense Val', linestyle='--', color='blue')
axes[0].plot(history_lstm.history['accuracy'], label='LSTM Train', linestyle='-', color='green')
axes[0].plot(history_lstm.history['val_accuracy'], label='LSTM Val', linestyle='--', color='green')
axes[0].set_title('Training Accuracy Comparison')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(history_dense.history['loss'], label='Dense Train', linestyle='-', color='blue')
axes[1].plot(history_dense.history['val_loss'], label='Dense Val', linestyle='--', color='blue')
axes[1].plot(history_lstm.history['loss'], label='LSTM Train', linestyle='-', color='green')
axes[1].plot(history_lstm.history['val_loss'], label='LSTM Val', linestyle='--', color='green')
axes[1].set_title('Training Loss Comparison')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
def predict_message(text, model, model_name):
    """Predict if a message is spam or ham"""
    # Preprocess
    cleaned = preprocess_text(text)
    
    # Tokenize and pad
    sequence = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(sequence, maxlen=MAX_LEN, padding='post', truncating='post')
    
    # Predict
    prediction = model.predict(padded, verbose=0)[0][0]
    label = 'SPAM' if prediction > 0.5 else 'HAM'
    confidence = prediction if prediction > 0.5 else 1 - prediction
    
    print(f"\n{model_name} Model:")
    print(f"Message: {text[:100]}...")
    print(f"Prediction: {label} (Confidence: {confidence:.2%})")
    
    return label, confidence

# Test samples
test_messages = [
    "Congratulations! You've won a $1000 gift card. Click here to claim now!",
    "Hey, are we still meeting for lunch tomorrow at noon?",
    "URGENT: Your account has been suspended. Verify your identity immediately.",
    "Thanks for the meeting notes. I'll review them and get back to you.",
    "Free entry in 2 a weekly comp to win iPhone. Text WIN to 12345 now!"
]

print("="*70)
print("TESTING WITH SAMPLE MESSAGES")
print("="*70)

for i, msg in enumerate(test_messages, 1):
    print(f"\n{'='*70}")
    print(f"Test Message {i}:")
    print(f"{'='*70}")
    
    # Predict with both models
    dense_label, dense_conf = predict_message(msg, dense_model, "Dense")
    lstm_label, lstm_conf = predict_message(msg, lstm_model, "LSTM")

## 16. Nhận xét và phân tích kết quả

### 📊 Kết quả chi tiết:

#### **1. Mô hình Dense Network:**
- **Accuracy:** 97.77% ⭐⭐⭐⭐⭐
- **Precision (Ham):** 99% - Rất tốt trong việc nhận diện tin nhắn không phải spam
- **Precision (Spam):** 95% - Tốt trong việc phát hiện spam
- **Recall (Ham):** 98% - Ít bỏ sót tin nhắn ham
- **Recall (Spam):** 96% - Phát hiện được hầu hết spam
- **F1-Score:** 0.97-0.99 (rất cân bằng)

**Confusion Matrix Dense:**
- True Ham: 2313, False Spam: 41 (chỉ 1.7% ham bị nhận nhầm)
- True Spam: 762, False Ham: 29 (chỉ 3.7% spam bị bỏ sót)

---

#### **2. Mô hình LSTM:**
- **Accuracy:** 80.51% ⭐⭐⭐
- **Precision (Ham):** 87%
- **Precision (Spam):** 61% - Thấp hơn đáng kể
- **Recall (Ham):** 87%
- **Recall (Spam):** 62% - Bỏ sót nhiều spam
- **F1-Score:** 0.62-0.87 (không cân bằng)

**Confusion Matrix LSTM:**
- True Ham: 2042, False Spam: 312 (13.2% ham bị nhận nhầm - cao!)
- True Spam: 490, False Ham: 301 (38% spam bị bỏ sót - rất cao!)

---

### 🔍 Phân tích nguyên nhân:

#### **Tại sao Dense model tốt hơn LSTM (97.77% vs 80.51%)?**

**1. Đặc điểm dữ liệu:**
- Spam detection dựa nhiều vào **từ khóa đặc trưng** ("free", "win", "congratulations", "claim", "urgent")
- **Không cần context tuần tự** - chỉ cần biết có xuất hiện từ spam hay không
- Dense model với GlobalMaxPooling1D bắt được các từ khóa quan trọng nhất

**2. Kiến trúc phù hợp:**
- **Dense model:** Nhẹ, nhanh, đơn giản - phù hợp với bag-of-words features
- **LSTM model:** Phức tạp hơn, cần học sequential patterns - **overkill** cho bài toán này
- LSTM có thể bị **underfitting** do:
  - Training chỉ 5 epochs (stopped early)
  - Cần nhiều data và epochs hơn để converge

**3. Training behavior:**
- **Dense:** Converge nhanh (epoch 2 đã đạt ~99% train accuracy), stable
- **LSTM:** Học chậm hơn, validation loss dao động (30% cuối cùng), chưa converge tốt

**4. Overfitting:**
- **Dense:** Val accuracy = 98.5%, Train accuracy = 100% → generalization tốt
- **LSTM:** Val accuracy cao hơn train accuracy ban đầu, sau đó dao động → không ổn định

---

### ✅ Kết luận:

**Mô hình Dense Network là lựa chọn tốt nhất** cho bài toán spam detection này vì:

1. ✅ **Hiệu suất cao:** 97.77% accuracy
2. ✅ **Precision/Recall cân bằng:** Cả 2 class đều > 95%
3. ✅ **Training nhanh:** Chỉ 8 epochs, mỗi epoch < 1 giây
4. ✅ **Đơn giản, dễ deploy:** Ít parameters hơn LSTM
5. ✅ **Robust:** Không overfitting, validation loss stable

**LSTM không phù hợp** trong trường hợp này vì:
- ❌ Quá phức tạp cho bài toán keyword-based
- ❌ Training chậm (61ms/step vs 1ms/step của Dense)
- ❌ Kết quả kém hơn nhiều (80% vs 97%)
- ❌ Bỏ sót 38% spam (301/791) - không chấp nhận được

---

### 💡 Khuyến nghị:

**Sử dụng Dense model** cho production vì:
- High accuracy + Fast inference
- Low false positive rate (chỉ 1.7%)
- Low false negative rate (chỉ 3.7%)

**Để cải thiện LSTM** (nếu muốn thử):
- Train nhiều epochs hơn (20-30 epochs)
- Tăng batch size
- Thử learning rate khác
- Thêm data augmentation
- Nhưng vẫn khó vượt qua Dense model cho bài toán này!